In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf # tensolflow 1.x 버전을 사용한다.
tf.disable_v2_behavior() # tensorflow 2.x 버전의 문법을 사용하지 않겠다고 선언한다. => 1.x 버전을 사용할 시에 코딩
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# x => [공부 시간, 과외 시간]
x = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]] # 7행 2열
print(x)
print(type(x))

xData = np.array(x)
print(xData)
print(type(xData))

[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
<class 'list'>
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


In [3]:
# y => [합격 여부] => 실제값
y = [0, 0, 0, 1, 1, 1, 1] # 1행 7열
print(y)
print(type(y))

yData = np.array(y)
print(yData)
print(type(yData))

# reshape(): numpy에서 데이터는 그대로 유지한채 배열의 형태(차원)을 변경한다.
# 1행 7열 numpy 배열 => 7행 1열 numpy 배열
yData = np.array(y).reshape(7, 1)
print(yData)
print(type(yData))

[0, 0, 0, 1, 1, 1, 1]
<class 'list'>
[0 0 0 1 1 1 1]
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


xData와 yData를 저장할 placeholder를 만든다.

In [4]:
# placeholder에 넘파이 배열을 대입하는 경우 shape 속성을 이용해서 대입될 넘파이 배열의 차원을 지정해야 한다.
# [None, 2] => placeholder에 대입될 넘파이 배열의 행의 개수는 몇개라도 상관없고 열의 개수는 무조건 2개라는 의미이다.
X = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # yData가 대입될 placeholder

가중치와 y절편을 임의의 값으로 정한다.

In [5]:
a = tf.Variable(tf.random_uniform([2, 1], dtype=tf.float32)) # 난수를 2행 1열로 발생시킨다.
b = tf.Variable(tf.random_uniform([1], dtype=tf.float32)) 
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a\n', sess.run(a), sep='')
print('b\n', sess.run(b), sep='')
print('=' * 80)
print('a1: {:f}, a2: {:f}, b: {:f}'.format(sess.run(a)[0][0], sess.run(a)[1][0], sess.run(b)[0]))
print('=' * 80)
print('a1: %f, a2: %f, b: %f' % (sess.run(a)[0], sess.run(a)[1], sess.run(b)))

a
[[0.05585134]
 [0.44677234]]
b
[0.89848363]
a1: 0.055851, a2: 0.446772, b: 0.898484
a1: 0.055851, a2: 0.446772, b: 0.898484


시그모이드 방정식, 오차 함수, 경사 하강

In [6]:
# sigmoid(): tensorflow에서 시그모이드 방정식을 계산한다.
# matmul(): tensorflow에서 행렬의 곱(적)을 계산한다.
# Y = 1 / (1 + np.e ** -(a1 * x1 + a2 * x2 + b))
y = tf.sigmoid(tf.matmul(X, a) + b) # 시그모이드 방정식, 예측값
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y)) # 오차 함수
gratident_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss) # 경사 하강

sigmoid() 메소드의 실행 결과(y)가 0.5 이상이면 1을, 0.5 미만이면 -1을 리턴시킨다.

In [7]:
sess = tf.Session()
# cast(data, dtype): tensorflow에서 데이터(data)를 dtype으로 형변환한다.
predict = tf.cast(tf.constant([1.9, 2.1]), dtype=tf.int32)
print(sess.run(predict)) # [1 2]
# cast() 메소드는 캐스팅할 데이터가 boolean 타입이면 True는 1로 False는 0으로 형변환한다.
predict = tf.cast(tf.constant([True, False]), dtype=tf.int32)
print(sess.run(predict)) # [1 0]
predict = tf.cast([0.5 >= 0.5, 0.5 < 0.5], dtype=tf.int32)
print(sess.run(predict)) # [1 0]

[1 2]
[1 0]
[1 0]


sigmoid() 메소드를 실행한 예상값(0 또는 1)을 계산한다.

In [10]:
predict = tf.cast(y >= 0.5, dtype=tf.float32)

예측값(predict)과 실제값(Y)이 일치하는 정도(정확도, accuracy)를 계산한다.

In [11]:
sess = tf.Session()
# equal(): 인수로 지정한 값이 같으면 True, 다르면 False를 리턴한다.
print(sess.run(tf.equal(1, 1)), sess.run(tf.equal(1, 0))) # True False
# 정확도 : 예측값과 실제값 비교 후 T/F를 1/0으로 캐스팅하고 평균을 계산한다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.int32)) # 예측값 vs 실제값

True False


학습시킨다.

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(3001):
    # sess.run([실행할 내용, ...], feed_dict={플레이스홀더 이름: 플레이스홀더에 저장할 데이터, ...})
    a_, b_, loss_, _ = sess.run([a, b, loss, gratident_descent], feed_dict={X: xData, Y: yData})
    if epoch % 300 == 0:
        print('epoch: {:6d}, loss: {:7.4f}, a1: {:7.4f}, a2: {:7.4f}, b: {:7.4f}'.format(epoch, loss_, a_[0][0], a_[1][0], b_[0]))

epoch:      0, loss:  1.8060, a1:  0.0441, a2:  0.8441, b:  0.0196
epoch:    300, loss:  0.2435, a1:  0.6678, a2: -0.2749, b: -2.8224
epoch:    600, loss:  0.1769, a1:  0.7040, a2: -0.0682, b: -4.2080
epoch:    900, loss:  0.1400, a1:  0.6397, a2:  0.2132, b: -5.2152
epoch:   1200, loss:  0.1158, a1:  0.5554, a2:  0.4853, b: -6.0174
epoch:   1500, loss:  0.0985, a1:  0.4712, a2:  0.7326, b: -6.6865
epoch:   1800, loss:  0.0856, a1:  0.3930, a2:  0.9536, b: -7.2613
epoch:   2100, loss:  0.0757, a1:  0.3222, a2:  1.1509, b: -7.7654
epoch:   2400, loss:  0.0678, a1:  0.2586, a2:  1.3276, b: -8.2147
epoch:   2700, loss:  0.0613, a1:  0.2017, a2:  1.4865, b: -8.6199
epoch:   3000, loss:  0.0560, a1:  0.1505, a2:  1.6304, b: -8.9890
